In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import tensorflow as tf
print(tf.__version__)
import gc
#from talos.model.layers import hidden_layers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LocallyConnected1D, Conv2D, Reshape, Dropout, MaxPool2D, Flatten
from keras.callbacks import ModelCheckpoint, History, EarlyStopping
from keras.activations import relu, elu, linear
from keras import backend as K
from keras import regularizers
from keras.datasets import mnist
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split

from tqdm import trange

plt.style.use('seaborn-darkgrid')
mpl.rcParams['figure.figsize'] = [10.0, 4.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 18

2.0.0


Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X = np.concatenate((X_test, X_train), axis=0)
X = np.array(X/255.0, dtype=np.float32)
nr_datapoints = X.shape[0]; nr_pixels = X.shape[1];

Y = np.concatenate((y_test, y_train), axis=0)
Y_onehot = np_utils.to_categorical(Y)
X = np.reshape(X, (nr_datapoints, nr_pixels, nr_pixels, 1))

del X_train, y_train, X_test, y_test

In [4]:
#opti = optimizers.Adam(amsgrad=True, lr=1e-3)
#def nn_clf():
#    model = Sequential()
#    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
#    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
#    model.add(MaxPool2D(pool_size=(2,2)))
#    model.add(Dropout(0.25))
#    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
#    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
#    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#    model.add(Dropout(0.25))
#    model.add(Flatten())
#    model.add(Dense(256, activation = "relu"))
#    model.add(Dropout(0.5))
#    model.add(Dense(10, activation = "softmax"))
#    model.compile(optimizer=opti, loss="categorical_crossentropy",  metrics=["accuracy"])
#    return model
#nn_clf().summary()

In [5]:
#callbacks = [EarlyStopping(monitor="val_accuracy", min_delta=0.001, patience=10, verbose=1, restore_best_weights=True)]

In [6]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y_onehot, test_size=0.3)

#clf = KerasClassifier(build_fn=nn_clf, batch_size=1024, epochs=1, validation_data=(X_test, Y_test), callbacks=callbacks, verbose=1)
#history = clf.fit(X_train, Y_train)
#Y_pred = clf.predict(X_test)

In [7]:
#plt.figure(figsize=(10,4))
#plt.plot(history.history["val_accuracy"], c="crimson")
#plt.plot(history.history["accuracy"], c="navy")
#plt.xlabel("Epochs")
#plt.ylabel("Accuracy")

In [8]:
def permute_params(params, nr_params=9, prob=0.3):
    to_permute_or_not_to_permute = np.random.uniform(0, 1, nr_params) < prob
    while(np.sum(to_permute_or_not_to_permute) < 1):
        to_permute_or_not_to_permute = np.random.uniform(0, 1, nr_params) < prob
    print(np.argwhere(to_permute_or_not_to_permute>0))
    
    new_params = params.copy()
    if to_permute_or_not_to_permute[0]:
        new_params["lr"] = new_params["lr"]*2**np.random.normal(0, 2)

    if to_permute_or_not_to_permute[1]:
        new_params["batch_size"] = max(1, round(new_params["batch_size"]*2**np.random.normal(0, 1)))
    
    if to_permute_or_not_to_permute[2]:
        new_params["nr_Conv1"] = max(1, new_params["nr_Conv1"] + np.random.choice([-1, 1]))
    
    if to_permute_or_not_to_permute[3]:
        new_params["nr_Conv2"] = max(1, new_params["nr_Conv2"] + np.random.choice([-1, 1]))
    
    if to_permute_or_not_to_permute[4]:
        new_params["Conv_size"] = max(1, round(new_params["Conv_size"]*2**np.random.normal(0, 0.5)))

    if to_permute_or_not_to_permute[5]:
        new_params["nr_flat"] = max(1, new_params["nr_flat"] + np.random.choice([-1, 1]))
        
    if to_permute_or_not_to_permute[6]:
        new_params["flat_size"] = max(1, round(new_params["flat_size"]*2**np.random.normal(0, 0.5)))

    if to_permute_or_not_to_permute[7]:
        new_params["reg"] = new_params["reg"]*2**np.random.normal(0, 1)
    
    if to_permute_or_not_to_permute[8]:
        new_params["dropout"] = new_params["dropout"]*2**np.random.normal(0, 1)
        
    return new_params

In [9]:
def make_model(params):
    opti = optimizers.Adam(amsgrad=True, lr=params["lr"])
    def nn_clf():
        model = Sequential()
        model.add(Conv2D(filters = params["Conv_size"], kernel_size = (5,5), padding = "same", activation ="relu", kernel_regularizer=regularizers.l2(params["reg"]), input_shape = (28,28,1)))
        for i in range(params["nr_Conv1"] - 1):
            model.add(Conv2D(filters = params["Conv_size"], kernel_size = (5,5), padding = "same", activation ="relu", kernel_regularizer=regularizers.l2(params["reg"])))
        model.add(MaxPool2D(pool_size=(2,2), padding="same"))
        model.add(Dropout(params["dropout"]))

        for i in range(params["nr_Conv2"]):
            model.add(Conv2D(filters = 2*params["Conv_size"], kernel_size = (3,3), padding = 'same', activation ='relu', kernel_regularizer=regularizers.l2(params["reg"])))
        model.add(MaxPool2D(pool_size=(2,2), padding="same"))
        model.add(Dropout(params["dropout"]))

        model.add(Flatten())
        for i in range(params["nr_flat"]):
            model.add(Dense(params["flat_size"], activation = "relu", kernel_regularizer=regularizers.l2(params["reg"])))
            model.add(Dropout(params["dropout"]))

        model.add(Dense(10, activation = "softmax"))
        model.compile(optimizer=opti, loss="categorical_crossentropy",  metrics=["accuracy"])
        return model
    return nn_clf

In [10]:
params = {
    "accepted" : 1,
    "acc_score" : 0,
    "lr" : 1e-3,
    "batch_size" : 1024,
    "nr_Conv1" : 2,
    "nr_Conv2" : 2,
    "Conv_size" : 32,
    "nr_flat" : 2,
    "flat_size" : 256,
    "reg" : 0.01,
    "dropout" : 0.25
}

all_params = {
    "accepted" : [],
    "acc_score" : [],
    "lr" : [],
    "batch_size" : [],
    "nr_Conv1" : [],
    "nr_Conv2" : [],
    "Conv_size" : [],
    "nr_flat" : [],
    "flat_size" : [],
    "reg" : [],
    "dropout" : []
}
for key in all_params:
    all_params[key].append(params[key])

In [11]:
hp_df = pd.DataFrame(columns = params.keys())
hp_df.loc[0] = params
hp_df

,accepted,acc_score,lr,batch_size,nr_Conv1,nr_Conv2,Conv_size,nr_flat,flat_size,reg,dropout
0,1.0,0.0,0.001,1024.0,2.0,2.0,32.0,2.0,256.0,0.01,0.25


In [ ]:
nr_averages = 10
random_states = np.random.randint(99, 999999, nr_averages)

t0 = time.time()
highest_acc = 0
for k in range(1000):
    print(k)
    new_params = permute_params(params)
    while new_params == params:
        new_params = permute_params(params)

    print(params)
    print(new_params)
        
    acc = 0
    for j in range(nr_averages):
        X_train, X_test, Y_train_onehot, Y_test_onehot = train_test_split(X, Y_onehot, test_size=0.3, random_state=random_states[j])
        
        model = make_model(new_params)
        callbacks = [EarlyStopping(monitor="val_accuracy", min_delta=0.001, patience=10, verbose=1, restore_best_weights=True)]
        clf = KerasClassifier(build_fn=model, batch_size=new_params["batch_size"], epochs=100, validation_data=(X_test, Y_test_onehot), callbacks=callbacks, verbose=0)
        history = clf.fit(X_train, Y_train_onehot)
        Y_pred = clf.predict(X_test)
        Y_test = np.argmax(Y_test_onehot, axis=1)
        acc += np.sum(Y_pred == Y_test)/len(Y_pred)
        print("acc = ", acc)
        
        del history, clf, model, callbacks
        gc.collect()
        K.clear_session()
    
    acc /= nr_averages
    
    new_params["acc_score"] = acc
    if acc < highest_acc:
        new_params["accepted"] = 0
    else:
        new_params["accepted"] = 1
        params = new_params
        highest_acc = acc

    for key in new_params:
        all_params[key].append(new_params[key])
        
    print(all_params)
    hp_df = pd.DataFrame.from_dict(all_params)
    hp_df.to_pickle("../data/hp_df.pickle")
    print("time [hours] = ", (time.time() - t0)/3600)

0
[[0]
 [1]
 [5]
 [8]]
{'accepted': 1, 'acc_score': 0, 'lr': 0.001, 'batch_size': 1024, 'nr_Conv1': 2, 'nr_Conv2': 2, 'Conv_size': 32, 'nr_flat': 2, 'flat_size': 256, 'reg': 0.01, 'dropout': 0.25}
{'accepted': 1, 'acc_score': 0, 'lr': 0.0012740333004635004, 'batch_size': 1974, 'nr_Conv1': 2, 'nr_Conv2': 2, 'Conv_size': 32, 'nr_flat': 1, 'flat_size': 256, 'reg': 0.01, 'dropout': 0.21232732455882472}
